In [1]:
import os

In [2]:
%pwd

'/Users/omariko/Documents/GitHub/NLP-Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/omariko/Documents/GitHub/NLP-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [7]:
from TextSummarizer.constants import CONFIG_FILE_PATH
from TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )
        
        return model_evaluation_config

In [9]:
from transformers import (AutoModelForSeq2SeqLM,
                          AutoTokenizer)
from datasets import load_from_disk
import evaluate
import torch
import pandas as Pd
from tqdm import tqdm

/Users/omariko/Documents/GitHub/NLP-Text-Summarizer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-02 14:53:26,658: INFO: config: PyTorch version 2.6.0 available.]


In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range (0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_dataset(self, dataset, metric, model, tokenizer,
                                         batch_size = 16,
                                         device = "cuda" if torch.cuda.is_available() else "cpu",
                                         column_text = "article",
                                         column_summary = "highlights"):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total = len(article_batches)):
            inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")

            summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                       attention_mask = inputs["attention_mask"].to(device),
                                       length_penalty = 0.8, num_beams = 8, max_length = 128)
            
            decoded_summaries = [tokenizer.decode(summary, skip_special_tokens = True, clean_up_tokenization_spaces = True)
                                 for summary in summaries]
            
            metric.add_batch(predictions = decoded_summaries, references = target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge", "rouge", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load(rouge_names)

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = Pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-02-28 14:30:24,555: INFO: common: Yaml file: config/config.yaml loadad successfully!]
[2025-02-28 14:30:24,557: INFO: common: Created directory at: artifacts]
[2025-02-28 14:30:24,557: INFO: common: Created directory at: artifacts/model_evaluation]


OSError: Incorrect path_or_model_id: 'artifacts/model_trainer/tokenizer'. Please provide either the path to a local folder or the repo_id of a model on the Hub.